In [12]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.5 # 2aB
nH = 10
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto3g", verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
e = mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Thu Oct  2 10:22:05 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.

In [ ]:
# def thouless_trans(t1):
#     # U, s, V = np.linalg.svd(t1.T)
#     # S = np.zeros_like(t1.T, dtype=float)
#     # np.fill_diagonal(S, s)
#     # u_ai = U@S
#     # u_ji = V.T
#     Q, R = np.linalg.qr(t1)
#     u_ai = R.T
#     u_ji = Q
#     u = np.vstack((u_ji,u_ai))
#     return u

In [6]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'rhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [148]:
def thouless_trans(t1):
    q, r = np.linalg.qr(t1)
    u_ai = r.T
    u_ji = q
    u_occ = np.vstack((u_ji,u_ai))
    u, _, _ = np.linalg.svd(u_occ)
    return u
u = thouless_trans(10*mycc.t1)
mo_t = mf.mo_coeff @ u
mf.mo_coeff = mo_t

In [149]:
t2_0 = np.zeros(mycc.t2.shape)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(10*mycc.t1, t2_0, eris)
ecc_t2_0 = eccsd+mf.e_tot
print(ecc_t2_0,eccsd)

-4.594488769302737 -0.0004128755147767654


In [ ]:
# from ad_afqmc.lno.afqmc import LNOAFQMC
# thresh = 1e-10
# mfcc = LNOAFQMC(mf, thresh=thresh, frozen=None).set(verbose=5)
# mfcc.thresh_occ = thresh
# mfcc.thresh_vir = thresh
# mfcc.nblocks = 5
# mfcc.nwalk_per_proc = 5
# mfcc.lo_type = 'pm'
# mfcc.no_type = 'cim'
# mfcc.frag_lolist = '1o'
# mfcc.force_outcore_ao2mo = True
# mfcc.kernel()


******** <class 'ad_afqmc.lno.afqmc.afqmc.LNOAFQMC'> ********
nocc = 2, nmo = 4
max_memory 4000 MB (current use 170 MB)
thresh_occ = 1.000000e-10
thresh_vir = 1.000000e-10
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 2  nlo = 2
frag_atmlist = None
frag_lolist = [[0], [1]]
frag_wghtlist = [1. 1.]
frag_nonvlist = None
    CPU time for LO and fragment        0.28 sec, wall time      0.31 sec
ao2mo est mem= 0.00 MB  avail mem= 3819.19 MB
Lov is saved to /tmp/r20rltcu
    CPU time for transforming DF-MP2 integrals      0.02 sec, wall time      0.00 sec
    CPU time for Integral xform         0.02 sec, wall time      0.01 sec
    CPU time for make pno               0.00 sec, wall time      0.00 sec
Solving impurity problem
#
# Preparing LNO-AFQMC calculation
# Calculating Cholesky integrals
# Generating Cholesky Integrals
# frozen orbitals are 0
# local active orbitals are [0 1 2 3]
# local active

UnboundLocalError: local variable 'value1' referenced before assignment

In [ ]:
# u = lno_maker.thouless_trans(mycc.t1)
# mo_t = mf.mo_coeff @ u
# mf.mo_coeff = mo_t
from ad_afqmc import pyscf_interface, config, mpi_jax, driver
from ad_afqmc.lno_afqmc import lnoafqmc_runner
import numpy as np

config.setup_jax()
MPI = config.setup_comm()

pyscf_interface.prep_afqmc(mf,chol_cut=1e-10)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))
# init_walkers = trial.get_init_walkers(wave_data, prop.n_walkers, restricted=True)
# init_walkers = u.T @ init_walkers
# driver.afqmc(ham_data,ham,prop,trial,wave_data,sampler,observable,options,MPI,init_walkers)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
#
# Preparing AFQMC calculation
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (5, 5)
# Number of basis functions: 10
# Number of Cholesky vectors: 32
#
# Number of MPI ranks: 1
#
# norb: 10
# nelec: (5, 5)
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 2
# walker_type: rhf
# trial: rhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [153]:
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, prop, trial, wave_data)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)
prop_data['walkers'] = u.T @ prop_data['walkers']
prop_data = prop.init_prop_data(trial, wave_data, ham_data, prop_data['walkers'])
print(prop_data['e_estimate'])
print(mf.e_tot)
print(ecc_t2_0)

-4.5944887692788585
-4.59407589378796
-4.594488769302737


In [130]:
from jax import random
config.setup_jax()
MPI = config.setup_comm()

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
seed = options["seed"]
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [142]:
prop_data, (block_energy, block_weight) \
    = sampler._block_scan(prop_data,None,ham_data,prop,trial,wave_data)
print(block_energy)

-4.641291410806528


In [154]:
trial.calc_energy(prop_data['walkers'], ham_data, wave_data)

Array([-4.59448877+0.j], dtype=complex128)

In [32]:
from jax import numpy as jnp
from jax import vmap
def walker_rhf_energy1(walker, ham_data, trial_mo):
    h0, rot_h1, rot_chol = ham_data["h0"], ham_data["rot_h1"], ham_data["rot_chol"]
    ene0 = h0
    green_walker = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    ene1 = 2.0 * jnp.sum(green_walker * rot_h1)
    f = jnp.einsum("gij,jk->gik", rot_chol, green_walker.T, optimize="optimal")
    c = vmap(jnp.trace)(f)
    exc = jnp.sum(vmap(lambda x: x * x.T)(f))
    ene2 = 2.0 * jnp.sum(c * c) - exc
    return ene0+ene1+ene2

# def walker_rhf_energy2(walker, ham_data, trial_mo):
#     h0, h1, chol = ham_data["h0"], ham_data["h1"][0], ham_data["chol"]
#     chol = chol.reshape((-1, h1.shape[-1], h1.shape[-1]))
#     nocc = trial_mo.shape[1]
#     ene0 = h0
#     gf = (walker @ (jnp.linalg.inv(trial_mo.T.conj() @ walker)) @ trial_mo.T.conj())
#     ene1 = 2.0 * jnp.einsum('ia,ia->',gf[:nocc,nocc:],h1[:nocc,nocc:])
#     f = jnp.einsum("gij,jk->gik", chol[:,:nocc,:nocc], gf[:nocc,:nocc], optimize="optimal")
#     c = vmap(jnp.trace)(f)
#     exc = jnp.sum(vmap(lambda x: x * x.T)(f))
#     ene2 = 2.0 * jnp.sum(c * c) - exc
#     return ene0+ene1+ene2

def rhf_olp(rhf1,rhf2):
    '''<rhf1|rhf2>'''
    olp = jnp.linalg.det(rhf1.T.conj() @ rhf2) ** 2
    return olp

In [155]:
walker_rhf_energy1(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])

Array(-4.59448877+0.j, dtype=complex128)

In [ ]:
# hcore = mf.get_hcore()
# mo = mf.mo_coeff
# # h1e_mo = np.dot(mo.T,np.dot( hcore, mo))
# h1e_mo = mo.T @ hcore @ mo
# print(h1e_mo)

[[-1.39494247e+00 -1.52655666e-16  1.18481321e-01  4.44089210e-16]
 [-7.45931095e-17 -1.23539794e+00  0.00000000e+00 -9.29652546e-02]
 [ 1.18481321e-01  5.55111512e-17 -1.09350707e+00  1.52655666e-16]
 [ 1.66533454e-16 -9.29652546e-02  3.81639165e-16 -1.00930479e+00]]


In [ ]:
# ham_data['h1']-h1e_mo

Array([[[ 0.00000000e+00,  3.90312782e-17,  2.77555756e-17,
         -1.38777878e-16],
        [-3.90312782e-17,  0.00000000e+00,  2.77555756e-17,
          4.16333634e-17],
        [-2.77555756e-17, -2.77555756e-17,  0.00000000e+00,
          1.14491749e-16],
        [ 1.38777878e-16, -4.16333634e-17, -1.14491749e-16,
          0.00000000e+00]],

       [[ 0.00000000e+00,  3.90312782e-17,  2.77555756e-17,
         -1.38777878e-16],
        [-3.90312782e-17,  0.00000000e+00,  2.77555756e-17,
          4.16333634e-17],
        [-2.77555756e-17, -2.77555756e-17,  0.00000000e+00,
          1.14491749e-16],
        [ 1.38777878e-16, -4.16333634e-17, -1.14491749e-16,
          0.00000000e+00]]], dtype=float64)

In [166]:
trial.calc_energy(prop_data['walkers'], ham_data, wave_data)

Array([-4.59448877+0.j], dtype=complex128)

In [168]:
walker_rhf_energy1(u @ prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])\
    + walker_rhf_coexc(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])[0] \
    + walker_rhf_fock_energy(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])[0]

Array(-4.59448877+0.j, dtype=complex128)

In [ ]:
# chol = ham_data['chol']
# nao = ham_data['h1'].shape[-1]
# chol = chol.reshape((-1, nao, nao))
# print(chol.shape)
# eris_chol = jnp.einsum('gpr,gqs->prqs',chol,chol,optimize='optimal')
# print(eris_chol.shape)

(10, 4, 4)
(4, 4, 4, 4)


In [ ]:
# from pyscf import ao2mo
# from ad_afqmc import pyscf_interface
# eri_mo = ao2mo.kernel(mf.mol,mf.mo_coeff,compact=False)
# chol = pyscf_interface.modified_cholesky(eri_mo,max_error=chol_cut)

In [ ]:
# eri_mo = ao2mo.incore.full(mol.intor('int2e'), mf.mo_coeff)
# eri_mo = ao2mo.restore(1, eri_mo, mol.nao_nr())
# # eri_mo = eri_mo.reshape((nao, nao, nao, nao))
# print(eri_mo.shape)

(4, 4, 4, 4)


In [ ]:
# print(np.max(eri_mo-eris_chol))

1.5922312936410939e-16


In [25]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [26]:
from pyscf.ci.cisd import CISD
from pyscf.cc.ccsd import CCSD
from pyscf.cc.uccsd import UCCSD
from ad_afqmc.lno.cc import LNOCCSD
from ad_afqmc.lno_afqmc import lno_maker, afqmc_maker, data_maker, lnoafqmc_runner
from pyscf import ao2mo, mcscf, scf, lib

frozen = None
thresh = 1e-10
lo_type = 'boys'
run_frg_list = [0]
eris = None

mfcc = mf
if isinstance(mfcc, (CCSD, CISD)):
    full_cisd = True
    mf = mfcc._scf
else:
    full_cisd = False
    mf = mfcc

no_type = 'ie' # cim

if isinstance(thresh, list):
    thresh_occ, thresh_vir = thresh
else:
    thresh_occ = thresh*10
    thresh_vir = thresh

lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = lo_type
lno_cc.no_type = no_type
lno_cc.force_outcore_ao2mo = True

s1e = mf.get_ovlp()
lococc = lno_cc.get_lo(lo_type=lo_type) # localized active occ orbitals
# lococc,locvir = lno_maker.get_lo(lno_cc,lo_type) ### fix this for DF
if eris is None: eris = lno_cc.ao2mo()

frag_lolist = [[i] for i in range(lococc.shape[1])]
nfrag = len(frag_lolist)

frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [thresh_occ,thresh_vir]
print(f'# lno thresh {thresh_pno}')

if run_frg_list is None:
    run_frg_list = range(nfrag)
    
from jax import random
seeds = random.randint(random.PRNGKey(options["seed"]),
                    shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

for ifrag in run_frg_list:
    print(f'\n########### running fragment {ifrag+1} ##########')

    fraglo = frag_lolist[ifrag]
    orbfragloc = lococc[:,fraglo]
    THRESH_INTERNAL = 1e-10
    frzfrag, orbfrag, can_orbfrag \
        = lno_maker.make_lno(
            lno_cc,orbfragloc,THRESH_INTERNAL,thresh_pno
            )
    
    mol = mf.mol
    nocc = mol.nelectron // 2 
    nao = mol.nao
    actfrag = np.array([i for i in range(nao) if i not in frzfrag])
    # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
    actocc = np.array([i for i in range(nocc) if i in actfrag])
    actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
    nactocc = len(actocc)
    nactocc = len(actocc)
    nactvir = len(actvir)
    prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]

    print(f'# active orbitals: {actfrag}')
    print(f'# active occupied orbitals: {actocc}')
    print(f'# active virtual orbitals: {actvir}')
    print(f'# frozen orbitals: {frzfrag}')

    ecorr_cc,t1,t2 = lno_maker.lno_cc_solver(
                        mf,orbfrag,orbfragloc,frozen=frzfrag,eris=eris
                        )
    ci1 = np.array(t1)
    ci2 = t2 + lib.einsum("ia,jb->ijab",ci1,ci1)
    ci2 = ci2.transpose(0, 2, 1, 3)
    options["seed"] = seeds[ifrag]
    nelec_act = nactocc*2
    norb_act = nactocc+nactvir
    lnoafqmc_runner.prep_lnoafqmc_file(
            mf,orbfrag,options,
            norb_act=norb_act,nelec_act=nelec_act,
            prjlo=prjlo,norb_frozen=frzfrag,
            ci1=ci1,ci2=ci2,chol_cut=1e-10,
            )

lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228


macro= 1  f(x)= 101.97870152776  delta_f= 101.979  |g|= 58.3944  2 KF 10 Hx
macro= 2  f(x)= 97.22236209194  delta_f= -4.75634  |g|= 39.0553  2 KF 10 Hx
macro= 3  f(x)= 95.74602696065  delta_f= -1.47634  |g|= 30.6397  2 KF 10 Hx
macro= 4  f(x)= 95.24408659054  delta_f= -0.50194  |g|= 28.9266  2 KF 10 Hx
macro= 5  f(x)= 93.93065512168  delta_f= -1.31343  |g|= 25.158  2 KF 10 Hx
macro= 6  f(x)= 92.098062176497  delta_f= -1.83259  |g|= 25.5206  2 KF 10 Hx
macro= 7  f(x)= 88.614258845254  delta_f= -3.4838  |g|= 27.5866  2 KF 10 Hx
macro= 8  f(x)= 83.238466711799  delta_f= -5.37579  |g|= 31.2358  2 KF 10 Hx
macro= 9  f(x)= 76.125681206905  delta_f= -7.11279  |g|= 34.7645  2 KF 10 Hx
macro= 10  f(x)= 68.051949154684  delta_f= -8.07373  |g|= 36.0058  2 KF 10 Hx
macro= 11  f(x)= 60.071684107329  delta_f= -7.98027  |g|= 33.935  2 KF 10 Hx
macro= 12  f(x)= 53.187230658207  delta_f= -6.88445  |g|= 27.6938  2 KF 10 Hx
macro= 13  f(x)= 48.389230201603  delta_f= -4.798  |g|= 17.1913  2 KF 10 Hx
macro

In [27]:
from ad_afqmc import config, wavefunctions, stat_utils
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = (
    lnoafqmc_runner.prep_lnoafqmc_run())

config.setup_jax()
MPI = config.setup_comm()

# init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()  # Process rank
size = comm.Get_size()  # Total number of processes


### initialize propagation
seed = options["seed"]
propagator = prop
init_walkers = None
ham_data = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                                )._build_measurement_intermediates(ham_data, wave_data)
ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(ham_data, propagator, trial, wave_data)
prop_data = propagator.init_prop_data(trial, wave_data, ham_data, init_walkers)

prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data['e_estimate'])

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 10
# nelec: (5, 5)
# nchol: 38
#
# n_eql: 4
# n_prop_steps: 10
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 450
# walker_type: rhf
# trial: cisd
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# n_batch: 1
# ene0: 0
#
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
-5.016939861084169


In [77]:
calc_e_rhf = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                                )._calc_energy_restricted
cal_eorb_rhf = wavefunctions.rhf(trial.norb, trial.nelec,n_batch=trial.n_batch
                                )._calc_orbenergy

In [91]:
def walker_rhf_fock_energy(walker,ham_data,trial_mo):
    '''one-body part of rhf walker's energy with 
    non-orthonal trial C_ia <psi|H|psi_ia>
    should be (close to) zero for orthogonal trial!!!'''

    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
                    gw.T[nocc:,:nocc],optimize='optimal')
    tr_f = vmap(jnp.trace)(f)
    tr_chol = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    e_fock1 = 2*jnp.einsum('ia,ia->',gw[:nocc,nocc:],rot_h1[:nocc,nocc:])
    e_fock2 = 4*jnp.einsum('g,g->',tr_f,tr_chol)\
              - 2*jnp.einsum('gji,gij->',f,rot_chol[:,:nocc,:nocc])
    # ene12 = 2*jnp.sum(tf*tc) \
    #         - jnp.einsum('gik,gki->',f,rot_chol[:,:nocc,:nocc])
    return e_fock1+e_fock2,e_fock1,e_fock2

def walker_rhf_coexc(walker,ham_data,trial_mo):
    '''two-body part of rhf walker's energy with 
    non-orthonal trial C_ijab <psi|H|psi_{ia}^{jb}>'''

    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
                    gw.T[nocc:,:nocc],optimize='optimal')
    tr_f = vmap(jnp.trace)(f)
    e_col = 2*jnp.einsum('g,g->',tr_f,tr_f)
    e_exc = -jnp.einsum('gij,gji->',f,f)
    # e_col = 2*jnp.einsum('ia,jb,gia,gjb->',
    #                      gf[:nocc,nocc:],gf[:nocc,nocc:],
    #                      rot_chol[:,:nocc,nocc:],rot_chol[:,:nocc,nocc:])
    # e_exc = - jnp.einsum('ia,jb,gib,gja->',
    #                      gf[:nocc,nocc:],gf[:nocc,nocc:],
    #                      rot_chol[:,:nocc,nocc:],rot_chol[:,:nocc,nocc:])
    return e_col+e_exc,e_col,e_exc

def walker_norhf_energy(walker, ham_data, trial_mo):
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
                    gw.T[nocc:,:nocc],optimize='optimal')
    tr_f = vmap(jnp.trace)(f)
    # one-bady part
    tr_chol = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    e_fock1 = 2*jnp.einsum('ia,ia->',gw[:nocc,nocc:],rot_h1[:nocc,nocc:])
    e_fock2 = 4*jnp.einsum('g,g->',tr_f,tr_chol)\
              - 2*jnp.einsum('gji,gij->',f,rot_chol[:,:nocc,:nocc])
    ene1 = e_fock1+e_fock2
    # two-body part
    e_col = 2*jnp.einsum('g,g->',tr_f,tr_f)
    e_exc = -jnp.einsum('gij,gji->',f,f)
    ene2 = e_col+e_exc
    return ene1+ene2

In [ ]:
# # m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
# walker = prop_data['walkers'][0]
# trial_mo = wave_data['mo_coeff']
# rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
# nocc = rot_h1.shape[0]
# gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
# f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
#                 gw.T[nocc:,:nocc],optimize='optimal')
# # tr_f = vmap(jnp.trace)(f)
# tr_chol = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
# e_fock1 = 2*jnp.einsum('ia,ia->i',gw[:nocc,nocc:],rot_h1[:nocc,nocc:])
# e_fock2 = 4*jnp.einsum('gii,g->i',f,tr_chol)\
#             - 2*jnp.einsum('gij,gji->i',f,rot_chol[:,:nocc,:nocc])

In [ ]:
# print(e_fock1.sum())
# print(e_fock2.sum())
# print(e_fock1+e_fock2)
# print(sum(e_fock1+e_fock2))

(0.0008342048701999041-0.05914059703327663j)
(-0.0008342135997256271+0.05914058530018232j)
[-0.00098792-0.01761472j -0.00096544-0.01274369j -0.00066245-0.01029588j
  0.00159431+0.01077798j  0.0010215 +0.0298763j ]
(-8.729525723810694e-09-1.1733094298221891e-08j)


In [106]:
def walker_rhf_fock_eorb(walker,ham_data,wave_data):
    '''one-body part of rhf walker's energy with 
    non-orthonal trial C_ia <psi|H|psi_ia>
    should be (close to) zero for orthogonal trial!!!'''
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    trial_mo = wave_data['mo_coeff']
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
                    gw.T[nocc:,:nocc],optimize='optimal')
    # tr_f = vmap(jnp.trace)(f)
    tr_chol = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    e_fock1 = 2*jnp.einsum('ia,ka,ik->',gw[:nocc,nocc:],rot_h1[:nocc,nocc:],m)
    e_fock2 = 4*jnp.einsum('gik,ik,g->',f,m,tr_chol)\
              - 2*jnp.einsum('gij,gjk,ik->',f,rot_chol[:,:nocc,:nocc],m)
    return e_fock1+e_fock2

def walker_rhf_ceorb(walker,ham_data,wave_data):
    '''two-body part of rhf walker's energy with 
    non-orthonal trial C_ijab <psi|H|psi_{ia}^{jb}>'''
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    trial_mo = wave_data['mo_coeff']
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
                    gw.T[nocc:,:nocc],optimize='optimal')
    tr_f = vmap(jnp.trace)(f)
    e_col = 2*jnp.einsum('gik,ik,g->',f,m,tr_f)
    e_exc = -jnp.einsum('gij,gjk,ik->',f,f,m)
    return e_col+e_exc

def walker_norhf_orbe(walker,ham_data,wave_data):
    m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
    trial_mo = wave_data['mo_coeff']
    rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
    nocc = rot_h1.shape[0]
    gw = (walker.dot(jnp.linalg.inv(trial_mo.T.conj() @ walker))).T
    f = jnp.einsum('gja,ai->gji',rot_chol[:,:nocc,nocc:],
                    gw.T[nocc:,:nocc],optimize='optimal')
    # tr_f = vmap(jnp.trace)(f)
    tr_chol = vmap(jnp.trace)(rot_chol[:,:nocc,:nocc])
    e_fock1 = 2*jnp.einsum('ia,ka,ik->',gw[:nocc,nocc:],rot_h1[:nocc,nocc:],m)
    e_fock2 = 4*jnp.einsum('gik,ik,g->',f,m,tr_chol)\
              - 2*jnp.einsum('gij,gjk,ik->',f,rot_chol[:,:nocc,:nocc],m)
    ene1 = e_fock1+e_fock2
    tr_f = vmap(jnp.trace)(f)
    e_col = 2*jnp.einsum('gik,ik,g->',f,m,tr_f)
    e_exc = -jnp.einsum('gij,gjk,ik->',f,f,m)
    ene2 = e_col+e_exc
    return ene1+ene2

In [96]:
m = jnp.dot(wave_data["prjlo"].T,wave_data["prjlo"])
print(m.shape)

(5, 5)


In [112]:
isinstance(trial, (wavefunctions.cisd))

True

In [108]:
trial

cisd(norb=10, nelec=(5, 5), n_batch=1)

In [92]:
walker_rhf_fock_energy(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])

(Array(-8.72952573e-09-1.17330942e-08j, dtype=complex128),
 Array(0.0008342-0.0591406j, dtype=complex128),
 Array(-0.00083421+0.05914059j, dtype=complex128))

In [73]:
prop_data, (blk_en,blk_wt,blk_hf_orb_en,blk_cc_orb_en) \
    = afqmc_maker.block_orb(prop_data,ham_data,prop,trial,wave_data,sampler)
print(blk_en)
print(walker_rhf_energy1(prop_data['walkers'][0],ham_data,wave_data['mo_coeff']))
print(mf.e_tot)

-5.000420318017777
(-4.621558180328421+0.003094638741867148j)
-4.594330519246477


In [74]:
calc_e_rhf(prop_data['walkers'][0],ham_data,wave_data)

Array(-4.62155818+0.00309464j, dtype=complex128)

In [89]:
walker_rhf_ceorb(prop_data['walkers'][0],ham_data,wave_data)

Array(-0.01514809+0.00265803j, dtype=complex128)

In [94]:
walker_rhf_fock_eorb(prop_data['walkers'][0],ham_data,wave_data)

Array(0.0010215+0.0298763j, dtype=complex128)

In [107]:
walker_norhf_orbe(prop_data['walkers'][0],ham_data,wave_data)

Array(-0.01412659+0.03253433j, dtype=complex128)

In [93]:
walker_rhf_fock_energy(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])

(Array(-8.72952573e-09-1.17330942e-08j, dtype=complex128),
 Array(0.0008342-0.0591406j, dtype=complex128),
 Array(-0.00083421+0.05914059j, dtype=complex128))

In [86]:
rot_h1,rot_chol = ham_data['rot_h1'],ham_data['rot_chol']
walker = prop_data['walkers'][0]
print(cal_eorb_rhf(0,rot_h1,rot_chol,walker,wave_data,orbE=0))

(-0.015148090536907648+0.002658033773818848j)


In [76]:
mf.e_tot \
    + walker_norhf_energy(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])[0]

Array(-4.62155818+0.00309464j, dtype=complex128)

In [62]:
print(walker_rhf_fock_energy(prop_data['walkers'][0],ham_data,wave_data['mo_coeff']))
print(walker_rhf_coexc(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])[0])

(Array(-2.94908134e-08-2.88966882e-08j, dtype=complex128), Array(-0.00366678-0.07368464j, dtype=complex128), Array(0.00366675+0.07368461j, dtype=complex128))
(-0.05410457269969854+0.005013833702532937j)


In [66]:
mf.e_tot \
    + walker_rhf_coexc(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])[0] \
    + walker_rhf_fock_energy(prop_data['walkers'][0],ham_data,wave_data['mo_coeff'])[0]

Array(-4.64843512+0.0050138j, dtype=complex128)